# Chains

> Chains are workflows that execute LLM based functionality for agents (or as part of the infrastructure generally).  

In [1]:
#| default_exp chains

In [ ]:
#| hide
from nbdev.showdoc import *

In [4]:
#| export  
from langchain.prompts import PromptTemplate
from alhazen.models import instantiate_llms
import dataclasses
from enum import auto, Enum
from typing import List, Tuple, Any, Dict
import alhazen_resources
from importlib_resources import files
import yaml 


In [6]:
#| export


@dataclasses.dataclass
class TaskInstruction:
    """A class that provides structure for task instructions (to be converted to LangChain PromptTemplates)."""

    # The name of this instruction template
    name: str

    # The type of this instruction template
    instruction_type: str

    # System prompts
    system: str

    # Instruction prompts
    instruction: str
    
    # The input variables that this instruction template requires
    input_variables: List[str] = dataclasses.field(default_factory=list)

    def generate_prompt_template(self) -> PromptTemplate:
        return PromptTemplate(
            input_variables=self.input_variables, 
            template=self.system + '\n' +self.instruction
        )

    def generate_llama2_prompt(self) -> PromptTemplate:
        return PromptTemplate(
            input_variables=self.input_variables, 
            template='''[INST]
                <<SYS>>'''+self.system+'''<</SYS>>
                '''+self.instruction+'''
                [/INST]'''
        )

class TaskInstructionRegistry:
    """A class that stores all PromptTemplates available in the system."""

    # The name of this instruction template
    registry: Dict[str, TaskInstruction] = {}

    def register_new_instruction_template(self, dict:Dict[str, str], override: bool = True):
        # check if all required fields are present
        assert 'name' in dict, "name is required"
        assert 'system' in dict, "system is required"
        assert 'instruction' in dict, "instruction is required"
        assert 'input_variables' in dict, "input_variables is required"

        name = dict['name']
        system = dict['system']
        instruction = dict['instruction']
        input_variables = dict['input_variables']

        self.register_instruction_template( TaskInstruction(name, system, instruction, input_variables), override=override)

    def deregister_instruction_template(self, name: str)->TaskInstruction:
        instruction_template = self.registry.pop(name)
        return instruction_template

    def register_instruction_template(self, template: TaskInstruction, override: bool = False):
        """Register a new conversation template."""
        if not override:
            assert template.name not in self.registry, f"{template.name} has been registered."
        self.registry[template.name] = template

    def get_instruction_template(self, name: str) -> TaskInstruction:
        """Get a conversation template."""
        if name not in self.registry:
            raise ValueError(f"{name} has not been registered.")
        return self.registry[name].copy()

    def load_prompts_from_yaml(self, file_name: str):
        prompts_yaml = files(alhazen_resources).joinpath(file_name).read_text()
        prompts_dict = yaml.safe_load(prompts_yaml)
        for pname in prompts_dict:
            dct = prompts_dict[pname]
            dct['name'] = pname
            self.register_new_instruction_template(dct)

    def __str__(self) -> str:
        out = "Registered instruction templates:\n"
        for name in self.registry:
            out += '- %s'%(name) + '\n'
        return out

# A global registry for all instruction templates
global instructions
instructions = TaskInstructionRegistry()
instructions.load_prompts_from_yaml('tiab_prompts.yaml')

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()